In [10]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk
import re
from nltk.corpus import stopwords
import string
import os
import numpy as np

csv_collection = []
for dirname, _, filenames in os.walk('C:\\Users\skrPlays\Downloads\\tweets'):
    for filename in filenames:
        fullpath= os.path.join(dirname, filename)
        csv_collection.append(fullpath)

In [11]:
csv_collection

['C:\\Users\\skrPlays\\Downloads\\tweets\\0401_UkraineCombinedTweetsDeduped.csv.gzip',
 'C:\\Users\\skrPlays\\Downloads\\tweets\\0402_UkraineCombinedTweetsDeduped.csv.gzip',
 'C:\\Users\\skrPlays\\Downloads\\tweets\\0403_UkraineCombinedTweetsDeduped.csv.gzip',
 'C:\\Users\\skrPlays\\Downloads\\tweets\\0404_UkraineCombinedTweetsDeduped.csv.gzip',
 'C:\\Users\\skrPlays\\Downloads\\tweets\\0405_UkraineCombinedTweetsDeduped.csv.gzip',
 'C:\\Users\\skrPlays\\Downloads\\tweets\\0406_UkraineCombinedTweetsDeduped.csv.gzip',
 'C:\\Users\\skrPlays\\Downloads\\tweets\\0407_UkraineCombinedTweetsDeduped.csv.gzip',
 'C:\\Users\\skrPlays\\Downloads\\tweets\\0408_UkraineCombinedTweetsDeduped.csv.gzip',
 'C:\\Users\\skrPlays\\Downloads\\tweets\\0409_UkraineCombinedTweetsDeduped.csv.gzip',
 'C:\\Users\\skrPlays\\Downloads\\tweets\\0410_UkraineCombinedTweetsDeduped.csv.gzip',
 'C:\\Users\\skrPlays\\Downloads\\tweets\\0411_UkraineCombinedTweetsDeduped.csv.gzip',
 'C:\\Users\\skrPlays\\Downloads\\tweets\\0

In [12]:
df = pd.read_csv(csv_collection.pop(),compression = 'gzip', index_col=0,
                 dtype={'userid':int, 'username':object, 'acctdesc':object, 'location':object,
                        'following':int, 'followers':int, 'totaltweets':int, 'usercreatedts':object,
                        'tweetid':int, 'tweetcreatedts':object, 'retweetcount':int, 'text':object,
                        'hashtags':object, 'language':object, 'coordinates':object, 'favorite_count':int,
                        'extractedts':object})
for data in csv_collection:
    try:
        tmp = pd.read_csv(data, compression = 'gzip', index_col=0,
                          dtype={'userid':int, 'username':object, 'acctdesc':object, 'location':object,
                                 'following':int, 'followers':int, 'totaltweets':int, 'usercreatedts':object,
                                 'tweetid':int, 'tweetcreatedts':object, 'retweetcount':int, 'text':object,
                                 'hashtags':object, 'language':object, 'coordinates':object, 'favorite_count':int,
                                 'extractedts':object})
    except:
        tmp = pd.read_csv(data, index_col = 0)
    df = pd.concat([df, tmp], axis=0)

C:\Users\skrPlays\AppData\Local\Temp\ipykernel_15060\2102135557.py:9: DtypeWarning: Columns (18,21,24,25,28) have mixed types. Specify dtype option on import or set low_memory=False.
  tmp = pd.read_csv(data, compression = 'gzip', index_col=0,


In [ ]:
pd.read_csv(data, index_col = 0)

In [13]:
df.shape

(25719255, 28)

In [14]:
data = df[["tweetcreatedts", "username", "text", "language"]]
data = data[data["language"]=="en"]
data.isnull().sum()
data_dates =  pd.to_datetime(df['tweetcreatedts'],  errors='coerce', infer_datetime_format=True).dt.date
data_dates.value_counts()

2022-03-07    567745
2022-03-06    566767
2022-03-05    546780
2022-03-08    519385
2022-03-09    493857
               ...  
2022-04-16    332860
2022-04-17    325909
2022-04-27    318809
2022-04-25    317674
2022-02-24    298045
Name: tweetcreatedts, Length: 63, dtype: int64

In [15]:
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")
stopword=set(stopwords.words('english'))

def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
data["tweet_stemmed"] = data["text"].apply(clean)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\skrPlays\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
nltk.download('vader_lexicon')
sentiments = SentimentIntensityAnalyzer()
data["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in data["tweet_stemmed"]]
data["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in data["tweet_stemmed"]]
data["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in data["tweet_stemmed"]]
data["Compound"] = [sentiments.polarity_scores(i)["compound"] for i in data["tweet_stemmed"]]
data = data[["tweetcreatedts", "text", "Positive", "Negative", "Neutral", "Compound"]]
print(data.head())

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\skrPlays\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


               tweetcreatedts  \
0  2022-03-31 00:00:00.000000   
1  2022-03-31 00:00:00.000000   
2  2022-03-31 00:00:00.000000   
3  2022-03-31 00:00:00.000000   
4  2022-03-31 00:00:00.000000   

                                                text  Positive  Negative  \
0  Humanitarian drive in Midhurst sends medicine,...     0.213     0.000   
1  This morning, #LetUsPray for #RussiaUkraine th...     0.291     0.000   
2  #RussiaUkraineWar | Horrors of nuclear war hau...     0.000     0.668   
3    Moral support. #Ukraine https://t.co/wcysaBFdkE     0.574     0.000   
4  #Russian citizens KNOW the genocide going on i...     0.123     0.261   

   Neutral  Compound  
0    0.787    0.2263  
1    0.709    0.7506  
2    0.332   -0.9590  
3    0.426    0.4019  
4    0.616   -0.5719  


In [17]:
conditions = [
    data['Compound'] > 0,
    data['Compound'] == 0,
    data['Compound'] < 0,
]

values = ['Positive', 'Neutral', 'Negative']

data['Sentiment'] = np.select(conditions, values)
print(data['Sentiment'].value_counts())

Negative    8336566
Positive    5069514
Neutral     4078228
Name: Sentiment, dtype: int64


In [18]:
sentiment_per_ts = data[['tweetcreatedts','Compound','Sentiment']]
#sentiment_per_ts['tweetcreatedts'].value_counts()

sentiment_per_ts['tweet_date'] =  pd.to_datetime(sentiment_per_ts['tweetcreatedts'],  errors='coerce', infer_datetime_format=True).dt.date

C:\Users\skrPlays\AppData\Local\Temp\ipykernel_15060\1532893575.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentiment_per_ts['tweet_date'] =  pd.to_datetime(sentiment_per_ts['tweetcreatedts'],  errors='coerce', infer_datetime_format=True).dt.date


In [19]:
print(sentiment_per_ts.shape)
sentiment_per_ts.dropna(inplace=True)
print(sentiment_per_ts.shape)

(17484308, 4)
(17484308, 4)


C:\Users\skrPlays\AppData\Local\Temp\ipykernel_15060\1187916437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentiment_per_ts.dropna(inplace=True)


In [20]:
summary_sentiments_per_date=sentiment_per_ts.groupby('tweet_date')['Compound'].describe()
population_sentiments_per_date = sentiment_per_ts.groupby('tweet_date')['Sentiment'].value_counts(normalize=True).unstack(level=-1)
pd.merge(summary_sentiments_per_date,population_sentiments_per_date,on = 'tweet_date').to_csv("sentimentPerDate_en.csv")